# Audio Classification with CNN under Cross-Validation Experiment
Try to analysis the data with,
+ accuracy
+ precision
+ recall
  
both of training step and validation stage.
Meanwhile, try to use the plot-box to demonstrates the data.

# The Cross-Validation under Multi-Classification
`This explaination comes from the Chat-GPT4o.`
Calculating precision and recall for a multi-class classification algorithm involves considering each class separately and then aggregating the results. Here's how you can do it step by step:

### 1. Definitions

- **Precision** for a class $C_i$: The ratio of true positive predictions for class $C_i$ to the total number of instances predicted as $C_i$ (i.e., true positives + false positives).
  
  $$
  \text{Precision}(C_i) = \frac{\text{True Positives}(C_i)}{\text{True Positives}(C_i) + \text{False Positives}(C_i)}
  $$

- **Recall** for a class $C_i $: The ratio of true positive predictions for class $ C_i $ to the total number of instances that actually belong to $ C_i $ (i.e., true positives + false negatives).

  $$
  \text{Recall}(C_i) = \frac{\text{True Positives}(C_i)}{\text{True Positives}(C_i) + \text{False Negatives}(C_i)}
  $$

### 2. Confusion Matrix

For a multi-class classification problem, you typically generate a confusion matrix, where each row represents the actual class, and each column represents the predicted class. The element at position $ (i, j) $ in the confusion matrix represents the number of instances of class $ i $ that were predicted as class $ j $.

### 3. Calculation for Each Class

- **True Positives (TP)**: The diagonal element for the class $ C_i $ in the confusion matrix, i.e., $ \text{TP}(C_i) = \text{Confusion Matrix}[i][i] $.
- **False Positives (FP)**: The sum of the corresponding column $ i $ minus the true positives, i.e., $ \text{FP}(C_i) = \sum_j \text{Confusion Matrix}[j][i] - \text{Confusion Matrix}[i][i] $.
- **False Negatives (FN)**: The sum of the corresponding row $ i $ minus the true positives, i.e., $ \text{FN}(C_i) = \sum_j \text{Confusion Matrix}[i][j] - \text{Confusion Matrix}[i][i] $.

### 4. Precision and Recall for Each Class

Using the definitions and the components from the confusion matrix, you can calculate precision and recall for each class $ C_i $:

$$
\text{Precision}(C_i) = \frac{\text{TP}(C_i)}{\text{TP}(C_i) + \text{FP}(C_i)}
$$

$$
\text{Recall}(C_i) = \frac{\text{TP}(C_i)}{\text{TP}(C_i) + \text{FN}(C_i)}
$$

### 5. Aggregating Results

To get overall metrics, you can use macro-averaging or micro-averaging:

- **Macro-Averaging**: Calculate the metrics for each class independently and then take the average.

  $$
  \text{Macro Precision} = \frac{1}{N} \sum_{i=1}^{N} \text{Precision}(C_i)
  $$

  $$
  \text{Macro Recall} = \frac{1}{N} \sum_{i=1}^{N} \text{Recall}(C_i)
  $$

- **Micro-Averaging**: Aggregate the contributions of all classes to calculate the metrics. Essentially, you sum up the true positives, false positives, and false negatives across all classes and then calculate precision and recall.

  $$
  \text{Micro Precision} = \frac{\sum_{i=1}^{N} \text{TP}(C_i)}{\sum_{i=1}^{N} (\text{TP}(C_i) + \text{FP}(C_i))}
  $$

  $$
  \text{Micro Recall} = \frac{\sum_{i=1}^{N} \text{TP}(C_i)}{\sum_{i=1}^{N} (\text{TP}(C_i) + \text{FN}(C_i))}
  $$

### Example

Let's consider a simple example with three classes A, B, and C, and the following confusion matrix:

$$
\begin{array}{c|ccc}
    & \text{Pred A} & \text{Pred B} & \text{Pred C} \\
    \hline
    \text{Actual A} & 40 & 5 & 5 \\
    \text{Actual B} & 3 & 50 & 2 \\
    \text{Actual C} & 2 & 4 & 45 \\
\end{array}
$$

For class A:

- $\text{TP}(A) = 40$
- $\text{FP}(A) = 3 + 2 = 5$
- $\text{FN}(A) = 5 + 5 = 10$

$$
\text{Precision}(A) = \frac{40}{40 + 5} = \frac{40}{45} \approx 0.89
$$

$$
\text{Recall}(A) = \frac{40}{40 + 10} = \frac{40}{50} = 0.8
$$

You can calculate precision and recall similarly for classes B and C and then aggregate the results using macro-averaging or micro-averaging as needed.

In [1]:
import pandas as pd
from pathlib import Path 

datasource_path = Path.home()/'dataset'/'UrbanSound8K'

# read the metadata file
metadata_file_path = datasource_path/'metadata'/'UrbanSound8K.csv'
df = pd.read_csv(metadata_file_path)
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [2]:
df['path'] = '/fold' + df['fold'].astype(str) + '/' + df['slice_file_name'].astype(str)
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,path
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,/fold5/100032-3-0-0.wav
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,/fold5/100263-2-0-117.wav
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,/fold5/100263-2-0-121.wav
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,/fold5/100263-2-0-126.wav
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,/fold5/100263-2-0-137.wav


In [3]:
from lib.crossValUtils import calIndexes
from lib.wavDataUtil import WavDataset
audio_path = datasource_path/'audio'

audioDS = WavDataset(df= df, data_path= audio_path)

audio, class_id = audioDS[0]
audio.shape, class_id

(torch.Size([2, 64, 344]), tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]))

In [4]:
from lib.crossValUtils import calIndexes, switchFold

indexes = calIndexes(dataset=audioDS, n_flod=10)
train_ids, val_ids = switchFold(val_fold=2, indexes=indexes)

# train_ids = df[df['fold'] != 5].index.to_numpy()
# val_ids = df[df['fold'] == 5].index.to_numpy()

train_ids.shape, val_ids.shape

((7869,), (863,))

In [5]:
from lib.crossValUtils import SubsetDs

train_ds = SubsetDs(dataset=audioDS, indexes=train_ids)
val_ds = SubsetDs(dataset=audioDS, indexes=val_ids)

audio, class_id = val_ds[9]
audio.shape, class_id

(torch.Size([2, 64, 344]), tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]))

In [6]:
from torch.utils.data import DataLoader

train_dl = DataLoader(dataset=train_ds, batch_size=16, shuffle=True)
val_dl = DataLoader(dataset=val_ds, batch_size=16, shuffle=False)

feature, labels = next(iter(val_dl))
feature.shape, labels

(torch.Size([16, 2, 64, 344]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]))

In [7]:
import torch 
from lib.acModel import AudioClassifier

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AudioClassifier().to(device=device)

model

AudioClassifier(
  (conv1): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv): Sequential(
    (0): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (2): ReLU()
    (3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(32, 64, kernel_

In [8]:
from lib.acModel import Processor
Processor.training(model=model, train_dl=train_dl, num_epochs=2, device=device)

Epoch: 0, Loss:2.12, accuracy:34.46%
Epoch: 1, Loss:1.97, accuracy:49.40%
Training End!!


In [9]:
from lib.crossValUtils import ValidationRecord
import torch

vr = ValidationRecord(n_fold=10, label_size=10)

model.eval()
with torch.no_grad():
    for (feature, labels) in val_dl:
        feature, labels = feature.to(device), labels.to(device)
        outputs = model(feature)
        vr.noteRecord(outputs=outputs, labels=labels, val_fold=2, iter=0)

vr.calRecord()
records = vr.getRecord()
records[records['val_fold'] == 2]


/root/Test-time-Adaptation-in-AC/legacy/audio_classification/simple_cnn_example/lib/crossValUtils.py:83: RuntimeWarning: invalid value encountered in double_scalars
  self.records.loc[i, 'accuracy'] = TP / (TP + FP)
/root/Test-time-Adaptation-in-AC/legacy/audio_classification/simple_cnn_example/lib/crossValUtils.py:109: RuntimeWarning: invalid value encountered in double_scalars
  return TP / (TP + FP)
/root/Test-time-Adaptation-in-AC/legacy/audio_classification/simple_cnn_example/lib/crossValUtils.py:113: RuntimeWarning: invalid value encountered in double_scalars
  return TP / (TP + FN)
/root/Test-time-Adaptation-in-AC/legacy/audio_classification/simple_cnn_example/lib/crossValUtils.py:105: RuntimeWarning: invalid value encountered in double_scalars
  self.records.loc[i, 'accuracy'] = TP / TN


,iteration,val_fold,label,accuracy,precision,recall,TP,FP,FN,TN
22,0.0,2.0,0.0,0.615385,0.615385,0.406780,48.0,30.0,70.0,715.0
23,0.0,2.0,1.0,NaN,NaN,0.000000,0.0,0.0,44.0,819.0
24,0.0,2.0,2.0,0.362832,0.362832,0.455556,41.0,72.0,49.0,701.0
25,0.0,2.0,3.0,0.411348,0.411348,0.659091,58.0,83.0,30.0,692.0
26,0.0,2.0,4.0,0.653333,0.653333,0.597561,49.0,26.0,33.0,755.0
27,0.0,2.0,5.0,0.562500,0.562500,0.577982,63.0,49.0,46.0,705.0
28,0.0,2.0,6.0,NaN,NaN,0.000000,0.0,0.0,37.0,826.0
29,0.0,2.0,7.0,0.613636,0.613636,0.818182,81.0,51.0,18.0,713.0
30,0.0,2.0,8.0,0.675000,0.675000,0.778846,81.0,39.0,23.0,720.0
31,0.0,2.0,9.0,0.467391,0.467391,0.467391,43.0,49.0,49.0,722.0


In [10]:
records[records['val_fold']==5]

,iteration,val_fold,label,accuracy,precision,recall,TP,FP,FN,TN
55,0.0,5.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
56,0.0,5.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
57,0.0,5.0,2.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
58,0.0,5.0,3.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
59,0.0,5.0,4.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
60,0.0,5.0,5.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
61,0.0,5.0,6.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
62,0.0,5.0,7.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
63,0.0,5.0,8.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
64,0.0,5.0,9.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [11]:
Processor.inference(model=model, val_dl=val_dl, device=device)

Validation accuracy: 54.808806%, Validation sample size is: 863.0


In [13]:
from lib.crossValUtils import corss_validation_analysis
from torch.utils.data import Dataset

def training(dataset: Dataset, indexes):
    train_ds = SubsetDs(dataset=dataset, indexes=indexes)
    train_dl = DataLoader(dataset=train_ds, batch_size=16, shuffle=True)
    model = AudioClassifier().to(device=device)
    Processor.training(model=model, train_dl=train_dl, num_epochs=1, device=device)
    return model

def inference(model, iter: int, val_fold: int, dataset: Dataset, indexes, records: ValidationRecord):
    val_ds = SubsetDs(dataset=dataset, indexes=indexes)
    val_dl = DataLoader(dataset=val_ds, batch_size=16, shuffle=False)

    model.eval()
    with torch.no_grad():
        for (feature, labels) in val_dl:
            feature, labels = feature.to(device), labels.to(device)
            outputs = model(feature)
            records.noteRecord(outputs=outputs, labels=labels, val_fold=val_fold, iter=iter)

rc = corss_validation_analysis(dataset=audioDS, n_fold=10, label_size=10, tranFunc=training, inferFunc=inference, n_iter=2)
rc.calRecord()
rc_df = rc.getRecord()
rc_df.to_csv('CNN_AC_Cross_Val.csv', sep=',', header=True, encoding='utf-8', mode='w')

Epoch: 0, Loss:2.08, accuracy:39.10%
Training End!!
Epoch: 0, Loss:2.12, accuracy:34.66%
Training End!!
Epoch: 0, Loss:2.09, accuracy:38.22%
Training End!!
Epoch: 0, Loss:2.11, accuracy:35.37%
Training End!!
Epoch: 0, Loss:2.11, accuracy:35.72%
Training End!!
Epoch: 0, Loss:2.08, accuracy:38.65%
Training End!!
Epoch: 0, Loss:2.11, accuracy:35.89%
Training End!!
Epoch: 0, Loss:2.10, accuracy:37.77%
Training End!!
Epoch: 0, Loss:2.08, accuracy:39.57%
Training End!!
Epoch: 0, Loss:2.13, accuracy:33.74%
Training End!!
Epoch: 0, Loss:2.10, accuracy:37.60%
Training End!!
Epoch: 0, Loss:2.08, accuracy:39.80%
Training End!!
Epoch: 0, Loss:2.11, accuracy:35.59%
Training End!!
Epoch: 0, Loss:2.12, accuracy:34.85%
Training End!!
Epoch: 0, Loss:2.11, accuracy:35.55%
Training End!!
Epoch: 0, Loss:2.10, accuracy:37.44%
Training End!!
Epoch: 0, Loss:2.10, accuracy:37.51%
Training End!!
Epoch: 0, Loss:2.09, accuracy:37.90%
Training End!!
Epoch: 0, Loss:2.12, accuracy:35.39%
Training End!!
Epoch: 0, Lo